In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
from maccabee.constants import Constants

NameError: name 'Constants' is not defined

In [91]:
_ = Constants.Params.all()

{ 'DEFAULT_SPEC_PATH': '/home/jovyan/work/Maccabee/maccabee/parameters/default_parameter_specification.yml',
  'METRIC_LEVEL_SPEC_PATH': '/home/jovyan/work/Maccabee/maccabee/parameters/metric_level_parameter_specifications.yml',
  'ParamSchemaKeysAndVals': <class 'maccabee.constants.Constants.Params.ParamSchemaKeysAndVals'>,
  'SCHEMA': { 'ACTUAL_CONFOUNDER_ALIGNMENT': { 'description': 'Marginal '
                                                              'probability '
                                                              'that '
                                                              'a '
                                                              'covariate '
                                                              'in '
                                                              'the '
                                                              'transformed '
                                                              'covariate '
                     